Есть датфрейм с authors, text, annotation и title. Текст предобработан, стоп-слова удалены. У одной статьи может быть несколько авторов, тогда они записаны в одной ячейке authors и разделены запятой. 

Функции:
- для вычисления всех соавторов одного человека по ФИО, они могут быть записаны в разных ячейках датафрейма. 

- функция для векторизации текста.  

- функция для вычисления коэффициента схожести.

- функция для рекомендации новых соавторов. 

В функцию для рекомендации новых соавторов подаётся на вход ФИО автора. На выходе ФИО автора, названия всех статей в которых он участвовал, список его соавторов, список трёх рекомендуемых новых авторов/коллективов авторов, названия их статей и коэффициент близости. 

Ограничения:

- Нельзя рекомендовать автора самому себе 
- Нельзя рекомендовать автору его текущих соавторов. 

Особенности:

Необходимо предусмотреть ситуацию когда рекомендуется несколько человек, написавших одну и ту же статью, при совпадении коэффициента близости у новых соавторов необходимо объединять их ФИО через запятую, писать их общую статью один раз и на 2 и 3 месте писать новых рекомендуемых авторов. При совпадении 2 и 3 рекомендации также следует их объединять и на третье место выводить следующих по коэффициенту схожести коллектив авторов/автора.

In [7]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import numpy as np
import pprint
from gensim.models import Word2Vec

In [9]:
df = pd.read_excel('date_it_lemm.xlsx')

In [10]:
#df['authors'] = df['authors'].str.split(',') #авторов через запятую разделяем

In [13]:
def get_coauthors(df, author_name):
    coauthors = set()
    for authors_list in df['authors'].str.split(', '):
        if author_name in authors_list:
            coauthors.update(authors_list)
    coauthors.discard(author_name)  # удаляем самого автора
    return list(coauthors)

def calculate_similarity(tfidf_matrix, target_indices, all_indices):
    similarities = cosine_similarity(tfidf_matrix[target_indices], tfidf_matrix[all_indices])
    return similarities.max(axis=0)

In [15]:
#модель Word2Vec
def train_word2vec_model(texts, save_path='word2vec.model'):
    sentences = [text.split() for text in texts]
    model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
    model.save(save_path)  # Сохраняем модель на диск
    return model

texts = df['text'].tolist()  # Тексты для обучения
train_word2vec_model(texts, save_path='word2vec.model')

In [17]:
def load_word2vec_model(load_path='word2vec.model'):
    return Word2Vec.load(load_path)  # Загружаем модель с диска

In [19]:
def vectorize_text_with_word2vec(text, model):
    words = text.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)  #усреднение векторов
    else:
        return np.zeros(model.vector_size)  #нулевой вектор если слов нет

def vectorize_texts_with_word2vec(df, text_column='text', model_path='word2vec.model'):
    print("Загрузка модели Word2Vec...")
    word2vec_model = load_word2vec_model(model_path)
    
    print("Векторизация текстов...")
    vectors = df[text_column].apply(lambda x: vectorize_text_with_word2vec(x, word2vec_model))
    
    #список векторов в массив
    return np.array(vectors.tolist()), word2vec_model

In [21]:
def recommend_new_coauthors(df, author_name, model_path='word2vec.model'):
    target_mask = df['authors'].str.contains(author_name, regex=False)  #данные автора
    target_indices = df[target_mask].index.tolist()
    
    if not target_indices:
        return {"error": "Автор не найден"}
    
    current_coauthors = get_coauthors(df, author_name)  #текущие соавторы
    
    tfidf_matrix, _ = vectorize_texts_with_word2vec(df, text_column='text', model_path=model_path)
    
    all_indices = df.index.tolist()
    similarities = calculate_similarity(tfidf_matrix, target_indices, all_indices)  # Схожести

    results = pd.DataFrame({
        'index': all_indices,
        'similarity': similarities,
        'authors': df['authors'],
        'title': df['title']
    })
    
    results = results[
        (results['similarity'] < 1.0) &  #исключаем статьи автора
        (~results['authors'].isin([author_name]))  #исключаем соавторов
    ]
    
    #группировка по коэффициенту схожести
    grouped = results.groupby('similarity').agg({
        'authors': lambda x: ', '.join(sorted(set(', '.join(x).split(', ')))),
        'title': lambda x: ', '.join(sorted(set(x)))
    }).reset_index().sort_values('similarity', ascending=False)
    
    recommendations = []
    used_authors = set(current_coauthors + [author_name])  #формируем рекомендации
    
    for _, row in grouped.iterrows():
        candidates = [a.strip() for a in row['authors'].split(',') if a.strip() not in used_authors]
        if candidates:
            unique_candidates = []
            for candidate in candidates:
                if candidate not in used_authors:
                    unique_candidates.append(candidate)
                    used_authors.add(candidate)
            
            if unique_candidates:
                recommendations.append({
                    'authors': ', '.join(unique_candidates),
                    'titles': row['title'],
                    'similarity': round(row['similarity'], 2)
                })
        
        if len(recommendations) >= 3:
            break
    
    output = {
        "Автор": author_name,
        "Текущие соавторы": current_coauthors,
        "Статьи": df[target_mask]['title'].tolist(),
        "Рекомендуемые соавторы": recommendations[:3]
    }
    
    #результаты в файлы
    save_results_to_txt(author_name, current_coauthors, output["Статьи"], output["Рекомендуемые соавторы"], df)
    
    return output

In [23]:
def save_results_to_txt(author_name, current_coauthors, articles, recommendations, df):
    #создаем папку
    import os
    if not os.path.exists("output"):
        os.makedirs("output")

    #файл 1: информация об авторе, его соавторах и нулевой статье
    with open(f"output/{author_name}_info.txt", "w", encoding="utf-8") as f:
        f.write(f"Автор: {author_name}\n")
        f.write(f"Соавторы: {', '.join(current_coauthors)}\n\n")
        
        #статьи у автора
        if articles:
            try:
                first_article = df[df['title'] == articles[0]].iloc[0]
                f.write(f"Заголовок: {first_article['title']}\n")
                f.write(f"Аннотация: {first_article['annotation']}\n")
                f.write(f"Текст: {first_article['text']}\n")
            except IndexError:
                f.write("Нулевая статья не найдена.\n")
        else:
            f.write("Статьи автора не найдены.\n")

    #файлы 2-4: рекомендуемые статьи
    for i, rec in enumerate(recommendations[:3], start=1):
        with open(f"output/{author_name}_recommendation_{i}.txt", "w", encoding="utf-8") as f:
            f.write(f"Коэффициент схожести: {rec['similarity']}\n")
            f.write(f"Рекомендуемые авторы: {rec['authors']}\n\n")
            
            titles = rec['titles'].split(', ')
            for title in titles:
                try:
                    article = df[df['title'] == title].iloc[0]
                    f.write(f"Заголовок: {article['title']}\n")
                    f.write(f"Аннотация: {article['annotation']}\n")
                    f.write(f"Текст: {article['text']}\n\n")
                except IndexError:
                    f.write(f"Статья с заголовком '{title}' не найдена.\n\n")

In [27]:
import pprint

rec_list = recommend_new_coauthors(df, 'Большаков А. О.', model_path='word2vec.model')
pprint.pprint(rec_list)
print('\n Статьи сохранены в папку output')

Загрузка модели Word2Vec...
Векторизация текстов...
{'Автор': 'Большаков А. О.',
 'Рекомендуемые соавторы': [{'authors': 'Здитовец А. Л.',
                             'similarity': 0.99,
                             'titles': 'ОСНОВНЫЕ ТЕХНОЛОГИИ И ФРЕЙМВОРКИ ДЛЯ '
                                       'БЕКЕНД-РАЗРАБОТКИ НА JAVA'},
                            {'authors': 'Жидков Виталий Алексеевич',
                             'similarity': 0.98,
                             'titles': 'Сравнительный анализ SCADA-систем, '
                                       'применяемых в диспетчерских службах '
                                       'Белгородской энергосистемы'},
                            {'authors': 'Массель Людмила Васильевна, Черноусов '
                                        'Антон Владимирович',
                             'similarity': 0.98,
                             'titles': 'Интеграция унаследованных программных '
                                       'комплексо